In [ ]:
import tensorflow as tf

In [1]:
#CONSTRUCTION PHASE
n_steps = 28
n_inputs = 28
n_neurons = 150
n_outputs = 10

learning_rate = 0.001

#create input and output placeholders
#X has shape [    t = 0              t = 1          t = 28 (n_steps)
#             [[x1,x2,...,x28],[x1,x2,...,x28],...,[x1,x2,...,x28]]  instance 1
#             [[x1,x2,...,x28],[x1,x2,...,x28],...,[x1,x2,...,x28]]  instance 2
#             [[x1,x2,...,x28],[x1,x2,...,x28],...,[x1,x2,...,x28]]  instance 3
#             ...
#             [[x1,x2,...,x28],[x1,x2,...,x28],...,[x1,x2,...,x28]]  total instances in batch (None | Undefined)
#            ]
X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])

#Y is an integer 0 - 9
y = tf.placeholder(tf.int32, [None])

#create the cells and the layers of the network
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32)

#Create Softmax layer with XEntropy 
#We can use the 'states' here, as there is only one layer.
logits = tf.layers.dense(states, n_outputs)
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,logits=logits)

#Compute the loss, mean xentropy score
loss = tf.reduce_mean(xentropy)

#Set the optimizer
#Here we use Adaptive Momement Estimation which keeps track of an exponentially decaying average of past gradients
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)

#Compute accuracy
#Does the highest logit correspond to the target class?
#in_top_k returns a 1D tensor full of boolean values
correct = tf.nn.in_top_k(logits, y, 1)

#We cast to floating point numbers and then take the mean to compute an accuracy score
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()

In [2]:
#IMPORT DATASET
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("tmp/data/")
X_test = mnist.test.images.reshape((-1, n_steps, n_inputs))
y_test = mnist.test.labels

Extracting tmp/data/train-images-idx3-ubyte.gz
Extracting tmp/data/train-labels-idx1-ubyte.gz
Extracting tmp/data/t10k-images-idx3-ubyte.gz
Extracting tmp/data/t10k-labels-idx1-ubyte.gz


In [3]:
#TRAINING PHASE
#n_epochs = 100
#batch_size = 150

#with tf.Session() as sess:
#    init.run()
#    for epoch in range(n_epochs):
#        for iteration in range(mnist.train.num_examples // batch_size):
#            X_batch, y_batch = mnist.train.next_batch(batch_size)
#            X_batch = X_batch.reshape((-1, n_steps, n_inputs))
#            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
#        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
#        acc_test = accuracy.eval(feed_dict={X: X_test, y: y_test})
#        print(epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)
        
        

#TRAINING PHASE
n_epochs = 100
batch_size = 150

#open a TensorFlow Session
with tf.Session() as sess:
    init.run()
    
    #Epoch is a single pass through the entire training set, followed by testing of the verification set.
    for epoch in range(n_epochs):
        #Number of batches, here we exhaust the training set
        for iteration in range(mnist.train.num_examples // batch_size):   

            #obtain batch of specified batch size
            X_batch, y_batch = mnist.train.next_batch(batch_size) 
            
            #reshape the array to size n_steps * n_inputs
            X_batch = X_batch.reshape((-1, n_steps, n_inputs)) 
            
            #feed in the batch
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        
        #compute accuracy of RNN against Training Set and Test Set
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: X_test, y: y_test})
        print(epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)

0 Train accuracy: 0.946667 Test accuracy: 0.9266


KeyboardInterrupt: 